Estudiaremos en el presente Notebook las cotizaciones y las distintas carácterísticas que componen los ETFs de BlackRock, conocidos como iShares.
Todos ellos serán los cotizados en **USD-$**, para poder realizar comparativas y agrupaciones de forma más sencilla.

Debido a esta condición, añadida a un mínimo de 20 años desde su salida al mercado para tener un número considerable de registros, nos encontramos ante **87 ETFs** distintos para analizar.


# Data Project

## Dataset

Nos quedará un Dataset compuesto por el histórico de la capitalización de cada ETF de iShare, que añadiremos a nuestro entorno de desarrollo en la carpeta de *pre-processed*, donde incluiremos todos los csv.

Estos csv han sido generados despúes de un primer paso de darle calidad a nuestros datos a partir de un script de Python que transforma los datos de la carpeta *raw* a los csv de *pre-processed*, preparándolos para su uso en KDB+.

En nuestro mismo directorio, recibiremos otro csv, donde estará la información a cargar para la tabla Resume, también generado por Python.


  > Tabla **Historical**: Registra el histórico de cotización de los ETFs. 
  >  - ***date***: La fecha del registro del histórico.    
  > - ***nav_value***: El valor de la participación en esa fecha.    
  > - ***shares***: El nº de participaciones en el mercado del ETF en esa fecha.  
  > - ***ticker***: El ticker identificativo del ETF.

  > Tabla **Resume**: Se trata de la información general del los ETFs de iShare. 
  >  - ***ticker***: El identificativo de cada ETF de iShares.    
  > - ***inception_date***: La fecha a la que sale al mercado el ETF.    
  > - ***net_assets***: El total de capitalización del ETF en el momento de sacar los datos.  
  > - ***equity_class***: Muestra el tipo de activo que compone el ETF. *Large/Mid Cap - All Cap - Large Cap - Mid Cap - Small Cap*
  > - ***region***: Muestra la región de dónde son las empresas. *Asia Pacific - North America  - Europe - Latin America - Global - Middle East and Africa*
  > - ***market***: Muestra el tipo de mercado que tiene las empresas. *Developed - Emerging*

1. Crearemos en un primer momento las tablas vacías.
2. Importaremos desde los csv la información y la insertaremos en nuestras tablas.

In [1]:
 historical:([]date:`date$();nav_value:`float$();shares:`long$();ticker:`$())
 resume_etf:([ticker:`$()]inception_date:`date$();net_assets:`float$();equity_class:`$();region:`$();market:`$())
 @[`historical;`ticker;`g#];
 meta historical

## Data Processing - Filter and Quality
Nuestros datos están disponibles en la carpeta *Data*, pero se presentan en un formato de Excel y con información poco valiosa y muy dispersa para el estudio. Por tanto consideramos que se encuentran en un proceso raw en nuestro proyecto de Ingeniería de Datos.

Por ello contamos con 2 scripts a realizar, donde filtraremos, renombraremos y daremos calidad a los datos, para posteriormente poder estudiarlos de mejor manera.

Pero antes de ello realizaremos la instalación de las librerías correspondientes, actualizadas en el *requirements.txt*:

In [2]:
system "pip install -r requirements.txt"

"Requirement already satisfied: pandas in /home/juan/miniconda3/envs/kdb/lib/..
"Requirement already satisfied: openpyxl in /home/juan/miniconda3/envs/kdb/li..
"Requirement already satisfied: numpy>=1.22.4 in /home/juan/miniconda3/envs/k..
"Requirement already satisfied: python-dateutil>=2.8.2 in /home/juan/minicond..
"Requirement already satisfied: pytz>=2020.1 in /home/juan/miniconda3/envs/kd..
"Requirement already satisfied: tzdata>=2022.7 in /home/juan/miniconda3/envs/..
"Requirement already satisfied: et-xmlfile in /home/juan/miniconda3/envs/kdb/..
"Requirement already satisfied: six>=1.5 in /home/juan/miniconda3/envs/kdb/li..


In [3]:
system "python3 Data/script_etf_a_csv.py Data/RawExcels/ Data/iShares-UnitedStates2.xlsx"
system "python3 Data/script_resume_etf.py Data/iShares-UnitedStates2.xlsx"

"Se ha exportado el archivo CSV como 'IUSG'"
"Se ha exportado el archivo CSV como 'EFA'"
"Se ha exportado el archivo CSV como 'EWA'"
"Se ha exportado el archivo CSV como 'FXI'"
"Se ha exportado el archivo CSV como 'IMCG'"
"Se ha exportado el archivo CSV como 'IMCB'"
"Se ha exportado el archivo CSV como 'EWG'"
"Se ha exportado el archivo CSV como 'IWF'"
"Se ha exportado el archivo CSV como 'IYH'"
"Se ha exportado el archivo CSV como 'IMCV'"
"Se ha exportado el archivo CSV como 'EWZ'"
"Se ha exportado el archivo CSV como 'IWR'"
"Se ha exportado el archivo CSV como 'EWC'"
"Se ha exportado el archivo CSV como 'EWY'"
"Se ha exportado el archivo CSV como 'DVY'"
"Se ha exportado el archivo CSV como 'ISCG'"
"Se ha exportado el archivo CSV como 'EZU'"
"Se ha exportado el archivo CSV como 'EWU'"
"Se ha exportado el archivo CSV como 'IXN'"
"Se ha exportado el archivo CSV como 'ISCB'"
"Se ha exportado el archivo CSV como 'IDGT'"
"Se ha exportado el archivo CSV como 'EWD'"
..


"Se ha exportado el archivo CSV como 'Data/pre-processed/resume.csv'"


## Data Ingest

Procedemos a importar los archivos de los ETFs que se encuentran en la ruta : *Data/PreProcessed/etf/* y los cargaremos  para poder operar con ellos.

In [ ]:
csv_files: system "ls Data/PreProcessed/etf/*.csv";

Insertamos en nuestra tabla **'historical'**  usando un iterador y las características de programación funcional y su compilación de derecha a izquierda que nos aporta el lenguaje.

Importamos los datos del csv, actualizamos la columna ticker con el nombre que queremos y realizamos un upsert sobre la tabla.

In [ ]:
//Ingesta de HISTORICAL
{ ticker_name: ssr[ssr[; "Data/PreProcessed/etf/"; ""] x; ".csv"; ""]; 
    `historical upsert update ticker:`$ticker_name from ("DFJ"; enlist csv) 0: `$x;
 } each csv_files;
//Ingesta de RESUME
`resume_etf insert ("SDFSSS"; enlist csv) 0: `$"Data/PreProcessed/resume.csv";

In [ ]:
show historical

In [ ]:
select count i from historical where ticker=`EWJ

In [ ]:
count historical

# New Data and Metadata

Procederemos a crear columnas en nuestras tablas con intención de recopilar información sobre el histórico y poder analizar posteriormente con mayor información y datos.

## Daily Returns:
 Añadiremos la diferencia de precios diaria en la tabla de *Historical* para poder calcular la volatilidad total.

 Por ello, lo primero que haremos será ordenar por ticker y fecha paraque se calcule de manera correcta los daily returns.

In [ ]:
historical: `ticker`date xasc historical;
historical: update daily_retns: (nav_value-prev nav_value)%prev nav_value by ticker from historical
show historical 

## Volatility
Añadimos la volatilidad total por ticker a la tabla *Resume*.

Tomaremos la fórmula de la Volatilidad cómo: 
> Volatility = σ<sub>period</sub> * √T<sub>period</sub>

En mercados financieros los años se tratan de 252 días de mercados abiertos, por tanto:

 - Volatilidad Anual: 252 días
 - Volatilidad 6 meses: 126 días
 - Volatilidad 3 meses: 63 días

### Historical Volatility - 1 Year

In [ ]:
year_vol: select vol1y: dev daily_retns * sqrt[252] by ticker from historical where date > max date - 251

### Historical Volatility - 6 Months

In [ ]:
sixm_vol: select  vol6m: var daily_retns * sqrt 126 by ticker from historical where date > max date - 125

### Historical Volatility - 3 Months

In [ ]:
threem_vol: select vol3m: var daily_retns * sqrt 63 by ticker from historical where date > max date - 62

Finalmente añadimos a la table *resume_etf* por cada etf de las 3 volatilidades calculadas.

In [ ]:
meta resume_etf: ((resume_etf lj `ticker xkey year_vol) lj `ticker xkey sixm_vol) lj `ticker xkey threem_vol

## Moving Averages

Calcularemos las medias móviles de ciertos periodos:
 - 200 días
 - 200 meses


### SMA 200 Days

In [ ]:
historical: `ticker`date xdesc historical;
historical: update d200sma:mavg[200;nav_value] by ticker from historical

### SMA 200 Months

Usamos la ordenación descendente + first en el fby para coger el primer dato del mes (el último al estar ordenado descendentemente).

In [ ]:
historical: `ticker`date xdesc historical
show historical: update m200sma:mavg[200;nav_value] by ticker from historical where date = (first;date)fby `month$date

In [ ]:
//Vemos los días del mes que son seleccionados para calcular la sma mensual
select from historical where ticker = `EWA, m200sma <> 0N

## Period Performance
Calcularemos los rendimientos del etf en los siguientes periodos, en un valor porcentual(%):
- 5 años
- 1 año
- 6 meses

### Performance - 5 Years

In [ ]:
perf_5y: select perf_5y: 100*(first nav_value - last nav_value)%last nav_value by ticker 
    from select from historical where (date = max date) or (date = max date-5*365)

In [ ]:
performance: { [initial_date; period]
 period2: select from historical where (date = initial_date) or (date = initial_date - period*365); 
 select performance:100*(first nav_value - last nav_value)%last nav_value by ticker from period2
 }

In [ ]:
performance[2007.10.10;6]

### Performance - 3 Years

In [ ]:
perf_3y: select perf_3y: 100*(first nav_value - last nav_value)%last nav_value by ticker 
    from select from historical where (date = max date) or (date = max date- 3*365 - 1)

### Performance - 1 Year

In [ ]:
perf_1y: select perf_1y: 100*(first nav_value - last nav_value)%last nav_value by ticker 
    from select from historical where (date = max date) or (date = max date- 1*365)

Al final añadimos a la table *resume_etf* los datos calculados por cada etf de los 3 rendimientos.

In [ ]:
meta resume_etf: ((resume_etf lj `ticker xkey perf_5y) lj `ticker xkey perf_3y) lj `ticker xkey perf_1y

## Drawdowns

Calcularemos:
- 5 años
- 3 años
- 1 año

In [ ]:
drawdown: {[table; period]
    table: `ticker`date xasc table;
    max_values: select date, ticker, max_value: nav_value
        from table where date >= max date - period * 365, 
        nav_value = (max; nav_value) fby ticker;
    w: flip max_values[`date],'max table[`date];
    min_values: select date, ticker, min_value: nav_value from table;
    filter_data: wj [w; `ticker`date; max_values; (min_values; (min; `min_value))];
    select ticker,drawdown: 100 * (max_value - min_value) % min_value from filter_data
 }

### Drawdown - 5 Years

In [ ]:
drw_5y: delete drawdown from `ticker xcol update drw_5y: drawdown from drawdown[historical;5]

### Drawdown - 3 Years

In [ ]:
drw_3y: delete drawdown from `ticker xcol update drw_3y: drawdown from drawdown[historical;3]

### Drawdown - 1 Year

In [ ]:
drw_1y: delete drawdown from `ticker xcol update drw_1y: drawdown from drawdown[historical;1]

Finalmente añadimos a la table *resume_etf* los datos calculados por cada etf de los 3 drawdowns.

In [ ]:
meta resume_etf: ((resume_etf lj `ticker xkey drw_5y)lj `ticker xkey drw_3y) lj `ticker xkey drw_1y

## Inflows/Outflows

Calculamos las variaciones de valor total de los etf, con la ampliación/salida de dinero en el mercado del etf por la variación mensual de las participaciones en circulación.

In [ ]:
historical: `ticker`date xasc historical

In [ ]:
historical: update in_out_flow_per: 100 * (shares - prev shares) % prev shares,
    in_out_flow_val: nav_value * shares - prev shares
    by ticker from historical where date = (last;date)fby `month$date

# Save to Data Warehouse

Una vez hemos generado toda la información y las tablas están listas, estas las guardaremos en disco.

Esto nos dará esa modularidad y persistencia de los datos para otros procesos futuros, pudiendo acudir a unos  datos ya elaborados y normalizados sin tener que acudir a los datos en bruto del DataLake. 

Guardaremos nuestras tablas en la carpeta de tablas del Data Warehouse.

In [ ]:
`:Data/DataWarehouse/Tables/historical set historical;
`:Data/DataWarehouse/Tables/resume_etf set resume_etf;

# Data Analysis: "The Ideal Conditions for ETFs to invest in!"

La idea es analizar los datos recopilados y poder acotar bajo que *parámetros/características* nuestros etf se comportan de cierto modo. 

El objetivo: **Poder hacer prediciones y modelos de las cotizaciones con mayores garantías.**

Nos encontramos ahora con unas tablas enriquecidas con mucha información de la siguiente manera:

> Tabla **Historical**: A parte de las ya existentes **date**, **nav_value**, **shares**, **ticker** se añade:
> - ***Daily Returns***: El aumento/pérdida de valor de manera % de un ETF respecto al dato previo.
> - ***SMA 200 Days***: El cálculo de la medía móvil de los últimos 200 días por cada ETF.
> - ***SMA 200 Months***: El cálculo de la medía móvil de los últimos 200 meses(último dato del mes disponible) por cada ETF.
> - ***Inflows/Outflows %***: Las entradas/salidas de dinero a nuestro ETF de manera mensual, con la ampliación/reducción de participaciones(shares) disponibles en el mercado.
> - ***Inflows/Outflows Value***: El valor en $ de la variación de participaciones en el mercado mensual.

> Tabla **Resume**: A parte de las ya existentes **ticker**, **inception_date**, **net_assets**, **equity_class**, **region**, **market** se añade:
> - ***Volatility: 1y, 6m, 3m***: La volatilidad de los precios del ETF en esos periodos.
> - ***Performance: 5y, 3y, 1y***: La variación del valor del ETF en esos periodos.
> - ***Drawdowns: 5y, 3y, 1y***; La caída máxima del precio de cotización en esos periodos.

# Extras + info futuro


In [ ]:
count select ticker from resume_etf where market = `$"Developed"
select from historical where ticker = `EWA, m200sma <> 0N
count select ticker from resume_etf where market = `Emerging

In [ ]:
//Filtrado  mezclando las dos tablas
select from historical where ticker in exec ticker from resume_etf where market = `$"Emerging"
//Lo mismo
show a: select ticker from resume_etf where market = `$"Developed"
select from historical where ticker in a[`ticker]

In [ ]:
select distinct equity_class from resume_etf
select distinct region from resume_etf

In [ ]:
//maximo aumento
select from historical where daily_retns=(max; daily_retns) fby ticker